# Data preparation

This notebook contains code that is used to prepare the MIMIC ICU cohort before starting the standardisation process

## Get DB Connection

In [ ]:
import os
import psycopg2


def getConnection():
    # Connect to postgres with a copy of the MIMIC-III database
    con = psycopg2.connect(
        dbname=os.environ['POSTGRES_DB_NAME'],
        user=os.environ['POSTGRES_USER_NAME'],
        host=os.environ['POSTGRES_HOSTNAME'],
        port=os.environ['POSTGRES_PORT_NUMBER'],
        password=os.environ['POSTGRES_PASSWORD']
        )
    return con

## Create a Cohort

In [ ]:
con = getConnection()


dropQuery = """drop table if exists omop_test_20220817.cohort_icd cascade"""
createQuery = '''
    create table omop_test_20220817.cohort_icd AS
    with stg1 as (
        select
        per.person_id,
        min(vis.visit_start_datetime) as anchor_time
        from
        omop_test_20220817.condition_occurrence con
        inner join omop_test_20220817.visit_occurrence vis
        on con.visit_occurrence_id = vis.visit_occurrence_id
        inner join mimiciv.admissions adm
        on adm.hadm_id = split_part(vis.visit_source_value, '|', 2)::int
        inner join mimiciv.patients pat
        on pat.subject_id = adm.subject_id
        inner join omop_test_20220817.person per
        on per.person_id = con.person_id
        where vis.visit_source_value not like '%-%'
        and (floor(date_part('day', vis.visit_start_datetime - make_timestamp(pat.anchor_year, 1, 1, 0, 0, 0))/365.0) + pat.anchor_age) > 18
        and (con.condition_source_value in ('99591', '99592', '78552', 'A419', 'R6520', 'R6521'))
        group by per.person_id
    )
    select
    s1.person_id,
    s1.anchor_time,
    vo.visit_occurrence_id
    from
    stg1 s1
    inner join omop_test_20220817.visit_occurrence vo
    on vo.person_id = s1.person_id
    and vo.visit_start_datetime = s1.anchor_time
'''

with con:
    with con.cursor() as cursor:
        cursor.execute(dropQuery)
        cursor.execute(createQuery)
